In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from scipy.sparse import hstack
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm_notebook, tqdm
from scipy import stats

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gc
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
def spearman_corr(y_true, y_pred):
        if np.ndim(y_pred) == 2:
            corr = np.mean([stats.spearmanr(y_true[:, i], y_pred[:, i])[0] for i in range(y_true.shape[1])])
        else:
            corr = stats.spearmanr(y_true, y_pred)[0]
        return corr

In [3]:
train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('test-2.csv').fillna(' ')
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [4]:
test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [9]:
np.unique(train['category'].values)

array(['CULTURE', 'LIFE_ARTS', 'SCIENCE', 'STACKOVERFLOW', 'TECHNOLOGY'],
      dtype=object)

In [10]:
train_text_1 = train['question_body']
test_text_1 = test['question_body']
all_text_1 = pd.concat([train_text_1, test_text_1])

train_text_2 = train['answer']
test_text_2 = test['answer']
all_text_2 = pd.concat([train_text_2, test_text_2])

train_text_3 = train['question_title']
test_text_3 = test['question_title']
all_text_3 = pd.concat([train_text_3, test_text_3])

In [12]:
sample_submission = pd.read_csv('sample_submission-2.csv').fillna(' ')
sample_submission.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [13]:
class_names = list(sample_submission.columns[1:])
class_names

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [14]:
class_names_q = class_names[:21]
class_names_a = class_names[21:]
class_names_a

['answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [15]:
class_names_2 = [class_name+'_2' for class_name in class_names]
for class_name in class_names:
    train[class_name+'_2'] = (train[class_name].values >= 0.5)*1

In [16]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=80000)
word_vectorizer.fit(all_text_1)
train_word_features_1 = word_vectorizer.transform(train_text_1)
test_word_features_1 = word_vectorizer.transform(test_text_1)

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=80000)
word_vectorizer.fit(all_text_2)
train_word_features_2 = word_vectorizer.transform(train_text_2)
test_word_features_2 = word_vectorizer.transform(test_text_2)

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=80000)
word_vectorizer.fit(all_text_3)
train_word_features_3 = word_vectorizer.transform(train_text_3)
test_word_features_3 = word_vectorizer.transform(test_text_3)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1, 4),
    max_features=47000)
char_vectorizer.fit(all_text_1)
train_char_features_1 = char_vectorizer.transform(train_text_1)
test_char_features_1 = char_vectorizer.transform(test_text_1)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1, 4),
    max_features=47000)
char_vectorizer.fit(all_text_2)
train_char_features_2 = char_vectorizer.transform(train_text_2)
test_char_features_2 = char_vectorizer.transform(test_text_2)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1, 4),
    max_features=47000)
char_vectorizer.fit(all_text_3)
train_char_features_3 = char_vectorizer.transform(train_text_3)
test_char_features_3 = char_vectorizer.transform(test_text_3)

train_features_1 = hstack([train_char_features_1, train_word_features_1, train_char_features_3, train_word_features_3])
test_features_1 = hstack([test_char_features_1, test_word_features_1, test_char_features_3, test_word_features_3])
train_features_2 = hstack([train_char_features_2, train_word_features_2])
test_features_2 = hstack([test_char_features_2, test_word_features_2])

CPU times: user 1min 28s, sys: 2.16 s, total: 1min 30s
Wall time: 1min 20s


In [17]:
train_features_1= train_features_1.tocsr()
train_features_2= train_features_2.tocsr()

In [18]:
alphas = {'question_asker_intent_understanding': 40,
         'question_body_critical':7,
          'question_conversational':35,
          'question_expect_short_answer':65,
          'question_fact_seeking':10,
          'question_has_commonly_accepted_answer':25,
          'question_interestingness_others':50,
          'question_interestingness_self':30,
          'question_multi_intent':7,
          'question_not_really_a_question':55,
          'question_opinion_seeking':15,
          'question_type_choice':4,
          'question_type_compare':30,
          'question_type_consequence':45,
          'question_type_definition':60,
          'question_type_entity':11,
          'question_type_instructions':6,
          'question_type_procedure':40,
          'question_type_reason_explanation':13,
          'question_type_spelling':1,
          'question_well_written':8,
          'answer_helpful':30,
          'answer_level_of_information':8,
          'answer_plausible':20,
          'answer_relevance':60,
          'answer_satisfaction':11,
          'answer_type_instructions':3,
          'answer_type_procedure':25,
          'answer_type_reason_explanation':3,
          'answer_well_written':25
         }

In [19]:
%%time
submission = pd.DataFrame.from_dict({'qa_id': test['qa_id']})

train_preds = []
test_preds = []
scores = []
spearman_scores = []

for class_name in tqdm_notebook(class_names_q):
    print(class_name)
    Y = train[class_name]
    
    n_splits = 3
    kf = KFold(n_splits=n_splits, random_state=47)

    train_oof_1 = np.zeros((train_features_1.shape[0], ))
    test_preds_1 = 0
    
    score = 0

    for jj, (train_index, val_index) in enumerate(kf.split(train_features_1)):
        #print("Fitting fold", jj+1)
        train_features = train_features_1[train_index]
        train_target = Y[train_index]

        val_features = train_features_1[val_index]
        val_target = Y[val_index]

        model = Ridge(alpha = alphas[class_name])
        model.fit(train_features, train_target)
        val_pred = model.predict(val_features)
        train_oof_1[val_index] = val_pred
        #print("Fold auc:", roc_auc_score(val_target, val_pred))
        #spearman_corr
        #score += roc_auc_score(val_target, val_pred)/n_splits

        test_preds_1 += model.predict(test_features_1)/n_splits
        del train_features, train_target, val_features, val_target
        gc.collect()
        
    model = Ridge(alpha = alphas[class_name])
    model.fit(train_features_1, Y)
    #print(test_preds_1.max())
    #print(test_preds_1.min())
    mms = MinMaxScaler(copy=True, feature_range=(0, 1))
    test_preds_1 = mms.fit_transform(test_preds_1.reshape(-1, 1)).flatten()
    preds = model.predict(test_features_1)
    mms = MinMaxScaler(copy=True, feature_range=(0, 1))
    preds = mms.fit_transform(preds.reshape(-1, 1)).flatten()
    submission[class_name] = (0.75*test_preds_1+0.25*preds+0.000005)/1.00001
    spearman_score = spearman_corr(train[class_name], train_oof_1)
    print("spearman_corr:", spearman_score) 
    spearman_scores.append(spearman_score)
    score = roc_auc_score(train[class_name+'_2'], train_oof_1)    
    print("auc:", score, "\n")
    train_preds.append(train_oof_1)
    test_preds.append(test_preds_1)
    scores.append(score)

question_asker_intent_understanding
('spearman_corr:', 0.35325156853346906)
('auc:', 0.6596484830805135, '\n')
question_body_critical
('spearman_corr:', 0.6218237635260194)
('auc:', 0.80552834514381, '\n')
question_conversational
('spearman_corr:', 0.382076831880739)
('auc:', 0.8774370804407055, '\n')
question_expect_short_answer
('spearman_corr:', 0.22990446465192632)
('auc:', 0.6385546131061819, '\n')
question_fact_seeking
('spearman_corr:', 0.31057791893684683)
('auc:', 0.6965143793353961, '\n')
question_has_commonly_accepted_answer
('spearman_corr:', 0.4034714039403396)
('auc:', 0.7975210842012358, '\n')
question_interestingness_others
('spearman_corr:', 0.3470866567089576)
('auc:', 0.6465157836647384, '\n')
question_interestingness_self
('spearman_corr:', 0.4819806907763744)
('auc:', 0.7692394005103035, '\n')
question_multi_intent
('spearman_corr:', 0.4796046843249426)
('auc:', 0.7761401398493758, '\n')
question_not_really_a_question
('spearman_corr:', 0.05434484312081326)
('auc:'

In [20]:
%%time

for class_name in tqdm_notebook(class_names_a):
    print(class_name)
    Y = train[class_name]
    
    n_splits = 3
    kf = KFold(n_splits=n_splits, random_state=47)

    train_oof_2 = np.zeros((train_features_2.shape[0], ))
    test_preds_2 = 0
    
    score = 0

    for jj, (train_index, val_index) in enumerate(kf.split(train_features_1)):
        #print("Fitting fold", jj+1)
        train_features = train_features_2[train_index]
        train_target = Y[train_index]

        val_features = train_features_2[val_index]
        val_target = Y[val_index]

        model = Ridge(alpha = alphas[class_name])
        model.fit(train_features, train_target)
        val_pred = model.predict(val_features)
        train_oof_2[val_index] = val_pred
        #print("Fold auc:", roc_auc_score(val_target, val_pred))
        #score += roc_auc_score(val_target, val_pred)/n_splits

        test_preds_2 += model.predict(test_features_2)/n_splits
        del train_features, train_target, val_features, val_target
        gc.collect()
        
    model = Ridge(alpha = alphas[class_name])
    model.fit(train_features_2, Y)
    print(test_preds_2.max())
    
    preds = model.predict(test_features_2)
    mms = MinMaxScaler(copy=True, feature_range=(0, 1))
    test_preds_2 = mms.fit_transform(test_preds_2.reshape(-1, 1)).flatten()
    mms = MinMaxScaler(copy=True, feature_range=(0, 1))
    preds = mms.fit_transform(preds.reshape(-1, 1)).flatten()
    submission[class_name] = (0.75*test_preds_2+0.25*preds+0.000005)/1.00001
        
    score = roc_auc_score(train[class_name+'_2'], train_oof_2) 
    
    
    spearman_score = spearman_corr(train[class_name], train_oof_2)
    print("spearman_corr:", spearman_score)
    print("auc:", score, "\n")
    spearman_scores.append(spearman_score)
    
    train_preds.append(train_oof_2)
    test_preds.append(test_preds_2)
    scores.append(score)

answer_helpful
0.9552368279405756
('spearman_corr:', 0.19624304400646306)
('auc:', 0.7298904737646458, '\n')
answer_level_of_information
0.7640376496043069
('spearman_corr:', 0.4020686263537812)
('auc:', 0.7233632450350717, '\n')
answer_plausible
0.9781353986721703
('spearman_corr:', 0.10594610279864893)
('auc:', 0.6000906543596506, '\n')
answer_relevance
0.9797275493434265
('spearman_corr:', 0.1327017174224555)
('auc:', 0.7131032778784385, '\n')
answer_satisfaction
0.9149172698025945
('spearman_corr:', 0.2618264224241846)
('auc:', 0.7889251948284717, '\n')
answer_type_instructions
1.162634645785389
('spearman_corr:', 0.6867820781666785)
('auc:', 0.8693860191504489, '\n')
answer_type_procedure
0.19184433968381265
('spearman_corr:', 0.22712648901852708)
('auc:', 0.6231349829759374, '\n')
answer_type_reason_explanation
1.0557589323860794
('spearman_corr:', 0.5732949616081158)
('auc:', 0.7996564270362659, '\n')
answer_well_written
0.9370084424795504
('spearman_corr:', 0.17449327769419437)

In [21]:
print("Mean auc:", np.mean(scores))
print("Mean spearman_scores", np.mean(spearman_scores))

('Mean auc:', 0.7705464846917921)
('Mean spearman_scores', 0.36278965064087226)


In [22]:
submission.to_csv('submission.csv', index=False)
submission.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.593357,0.455520,0.724251,0.157970,0.362661,0.262949,0.838206,0.717305,0.999995,...,0.683981,0.243750,0.232722,0.514244,0.224026,0.285824,0.377177,0.185401,0.463435,0.500517
1,46,0.618085,0.581916,0.242493,0.625234,0.475904,0.700012,0.540964,0.405039,0.312146,...,0.372141,0.700394,0.267475,0.808550,0.733578,0.697111,0.813476,0.504614,0.144305,0.257054
2,70,0.588835,0.539005,0.647609,0.393936,0.292062,0.275625,0.800482,0.678597,0.649853,...,0.647407,0.164005,0.134106,0.456728,0.192147,0.233545,0.311862,0.186233,0.507457,0.415933
3,132,0.320544,0.226931,0.181993,0.626251,0.382223,0.766550,0.308719,0.270877,0.324261,...,0.247393,0.450012,0.538491,0.528590,0.449425,0.591853,0.668773,0.543181,0.444103,0.394907
4,200,0.653402,0.503135,0.646148,0.562302,0.189838,0.340424,0.765007,0.731331,0.663830,...,0.529153,0.507041,0.558395,0.685899,0.395505,0.526787,0.315861,0.417680,0.524176,0.735129
